In [ ]:
# TODO Create an google api key and add it here
# TODO add the OPEN_AI_KEY from moodle
GOOGLE_API_KEY = ''
OPEN_AI_KEY = ''

In [ ]:
import requests

ADDRESS = "Greifenseestr. 53, 8050 Zürich"

# Convert address to URL format
address_url = ADDRESS.replace(" ", "+")

# Get latitude and longitude using Geocoding API
geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address_url}&key={GOOGLE_API_KEY}"
geo_response = requests.get(geocode_url).json()


if geo_response["status"] == "OK":
    location = geo_response["results"][0]["geometry"]["location"]
    lat, lon = location["lat"], location["lng"]

    # Angles to capture (90° = East, 180° = South, 270° = West)
    headings = [0, 90, 180, 270]  # 0° (North), 90° (East), 180° (South), 270° (West)

    for heading in headings:
        streetview_url = (
            f"https://maps.googleapis.com/maps/api/streetview?size=600x400&location={lat},{lon}"
            f"&heading={heading}&fov=80&key={GOOGLE_API_KEY}"
        )

        response = requests.get(streetview_url)

        if response.status_code == 200:
            filename = f"streetview_{heading}.jpg"
            with open(filename, "wb") as file:
                file.write(response.content)
            print(f"Street View image saved: {filename}")
        else:
            print(f"Error fetching the {heading}° Street View image:", response.status_code)
else:
    print("Error getting coordinates:", geo_response["status"])


In [ ]:
import base64
from openai import OpenAI

client = OpenAI(api_key=OPEN_AI_KEY)
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "streetview_0.jpg"
image_path_90 = "streetview_90.jpg"
# Getting the Base64 string
base64_image = encode_image(image_path)
base64_image_90 = encode_image(image_path_90)

prompt = "Analyze the road size, predict speed limit (options: 30, 50, 80), and determine house condition. Return as JSON."

completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": prompt },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image_90}",
                    },
                }
            ],
        }
    ],
)

print(completion.choices[0].message.content)